# XHARPy Example: L-Alanin with Quantum Espresso
 - Program: Quantum Espresso
 - PBE calculation
 - Constant core calculation

In [ ]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions,
                    refine, write_cif, write_res, write_fcf, add_density_entries_from_fcf)
import os

In [ ]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

### Loading Data

In [ ]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [ ]:
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [ ]:
hkl = shelxl_hkl2pd('iam.hkl')

In [ ]:
constraint_dict = lst2constraint_dict('iam.lst')

### Setting Options

In [ ]:
refinement_dict = {
    'f0j_source': 'qe',
    'reload_step': 1,
    'core': 'constant',
}

In [ ]:
computation_dict = {
    'symm_equiv': 'once',
    'mpicores': 2,
    'control': {
        'prefix': 'A23',
        'pseudo_dir': './pseudo/',
    },
    'system': {
        'ibrav': 8,
        'a': float(cell[0]),
        'b': float(cell[1]),
        'c': float(cell[2]),
        'ecutwfc': 50,
        'ecutrho': 200,
    },
    'paw_files': {
        'H': 'H.pbe-kjpaw_psl.1.0.0.UPF',
        'C': 'C.pbe-n-kjpaw_psl.1.0.0.UPF',
        'O': 'O.pbe-n-kjpaw_psl.1.0.0.UPF',
        'N': 'N.pbe-n-kjpaw_psl.1.0.0.UPF'
    },
    'k_points':{
        'mode': 'automatic',
        'input': '1 1 1 0 0 0'
    }
}

### Refinement

In [ ]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)

In [ ]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

### Writing Data to Disk

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, 'xharpy.cif'),
    cif_dataset='xharpy',
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, 'xharpy_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
add_density_entries_from_fcf(
    os.path.join(output_folder, 'xharpy.cif'),
    os.path.join(output_folder, 'xharpy_6.fcf')
)